In [1]:
import pymongo
import json
from threading import Thread


In [2]:
from datetime import datetime
datetime.now().time()

datetime.time(20, 43, 38, 533681)

In [5]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
#date parser
from dateutil import parser as date_parser

import random
user_agent  = UserAgent()

header ={"user-agent":user_agent.random}


In [6]:
urls = {"onlinekhabar":"http://english.onlinekhabar.com/feed",
        "the himalayan times":"https://thehimalayantimes.com/feed/",
        "setopati":"https://setopati.net/feed"}
for url in urls:
    print(urls[url])

http://english.onlinekhabar.com/feed
https://thehimalayantimes.com/feed/
https://setopati.net/feed


In [7]:
url = "https://setopati.net/feed"
response =  requests.get(url,headers=header)


In [8]:
response

<Response [200]>

In [89]:
url  = "https://thehimalayantimes.com/feed/"
# url = "https://setopati.net/feed"
response =  requests.get(url,headers)

pagesource = response.content
soup  = BeautifulSoup(pagesource,'xml')
item_list = soup.find_all('item')
# categorys=[]
print(soup)

# for post in item_list:
#     title = post.find('title')
#     pubdate = post.find('pubDate').text
#     category_list = post.find_all('category')
#     for category in category_list:
#         categorys.append(category)
#     description =  post.find('description').text

post  =  item_list[0]

def clean_description(description):
    soup_description =BeautifulSoup(description,'html.parser')
# print(soup_description.find('p'))
    if (soup_description.find('p')==None):
        c_description = soup_description
#     print('seto')
    else:
#     print('kalo')
        c_description = soup_description.find("p").text
    return c_description

d = post.find('description').text

x = (clean_description(d))
x

<?xml version="1.0" encoding="utf-8"?>
<html>
<head><title>403 Forbidden</title></head>
<body bgcolor="white">
<center><h1>403 Forbidden</h1></center>
<hr><center>nginx</center>
</hr>
</body>
</html>


IndexError: list index out of range

In [55]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

breakfastDB = myclient["breakfast"]
newsCollection = breakfastDB["allNews"]

def clean_description(description):
    soup_description =BeautifulSoup(description,'html.parser')
# print(soup_description.find('p'))
    if (soup_description.find('p')==None):
        c_description = soup_description.text
#     print('seto')
    else:
#     print('kalo')
        c_description = soup_description.find("p").text
    return c_description
    
# urls = ["http://english.onlinekhabar.com/feed","https://thehimalayantimes.com/feed/","https://setopati.net/feed"]
urls = {
    "onlinekhabar":"http://english.onlinekhabar.com/feed",
    "the himalayan times":"https://thehimalayantimes.com/feed/",
    "setopati":"https://setopati.net/feed"
    }

# final_df= pd.DataFrame()

for url in urls:
#     s_df= pd.DataFrame()
    try:
        response =  requests.get(urls[url],headers=header)
        pagesource = response.content
        soup  = BeautifulSoup(pagesource,'xml')
        item_list = soup.find_all('item')

        for post in item_list:

            categorys=[]
            aDic={}

            title = post.find('title').text # getting title
            pubdate = post.find('pubDate').text # getting publication data
            link  = post.find('link').text    # getting link of the article
            
            category_list = post.find_all('category') # category list
            for category in category_list:
                categorys.append(category.text)
            
            description =  post.find('description').text
            # cleaning the description 
            soup_description =BeautifulSoup(post.find('description').text,'html.parser')
            # print(soup_description)
#             if (soup_description.find('p')==None):
#                 description = soup_description
#             else:
#                 description = soup_description.find('p').text
            # cleaned
            aDic['Publisher']=url
            aDic['Title']=title
            aDic['Publish_Date'] = pubdate
            aDic['Link'] = link
            aDic['Category']=categorys
            aDic['Description']=clean_description(description)
            newsCollection.insert_one(aDic)
            # converting to dataframe 
           # instant_df = pd.DataFrame([aDic],columns=aDic.keys())
            #s_df = pd.concat([s_df,instant_df],ignore_index=True,axis=0,sort=False)
        #final_df = pd.concat([final_df,s_df],ignore_index=True,axis=0,sort=False)
    except Exception as e:
        print(e)
        pass

# final_df.to_json("all.json",orient='records')    
# final_df.to_json("index_all.json",orient='index') 



In [28]:
with open("index_all.json") as jsonfile:
    json_data  = json.load(jsonfile)

In [32]:
type(json_data)

dict

In [33]:
json_data

{'0': {'Publisher': 'onlinekhabar',
  'Title': 'Two Nepalis die on Everest and Makalu',
  'Publish_Date': 'Fri, 24 May 2019 10:56:11 +0000',
  'Link': 'http://english.onlinekhabar.com/two-nepalis-die-on-everest-and-makalu.html',
  'Category': ['Just In', 'Dhruba Bista', 'Nima Tenji Sherpa'],
  'Description': 'Kathmandu, May 24 Two more have died on two different 8,000 metre plus peaks of the country on Friday. Reputed mountain guide Dhruba Bista died at the base camp of Mount Everest after suffering from mountain sickness. Jiwan Bhattarai, an official at Himalaya Ecstasy Treks, says that Bista fell ill at Camp III and was […]'},
 '1': {'Publisher': 'onlinekhabar',
  'Title': 'China completes Durbar High School reconstruction in 10 months',
  'Publish_Date': 'Fri, 24 May 2019 10:34:06 +0000',
  'Link': 'http://english.onlinekhabar.com/china-completes-durbar-high-school-reconstruction-in-10-months.html',
  'Category': ['Heritage',
   'Just In',
   'Chinese support',
   'Durbar High Schoo

In [48]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

breakfastDB = myclient["breakfast"]
newsCollection = breakfastDB["allNews"]
try :
    x = newsCollection.insert_one(json_data)
except :
    pass


In [42]:
y = newsCollection.find_one()

In [43]:
print(y)

{'_id': ObjectId('5ceb51f0155c452670eaaa29'), '0': {'Publisher': 'onlinekhabar', 'Title': 'Two Nepalis die on Everest and Makalu', 'Publish_Date': 'Fri, 24 May 2019 10:56:11 +0000', 'Link': 'http://english.onlinekhabar.com/two-nepalis-die-on-everest-and-makalu.html', 'Category': ['Just In', 'Dhruba Bista', 'Nima Tenji Sherpa'], 'Description': 'Kathmandu, May 24 Two more have died on two different 8,000 metre plus peaks of the country on Friday. Reputed mountain guide Dhruba Bista died at the base camp of Mount Everest after suffering from mountain sickness. Jiwan Bhattarai, an official at Himalaya Ecstasy Treks, says that Bista fell ill at Camp III and was […]'}, '1': {'Publisher': 'onlinekhabar', 'Title': 'China completes Durbar High School reconstruction in 10 months', 'Publish_Date': 'Fri, 24 May 2019 10:34:06 +0000', 'Link': 'http://english.onlinekhabar.com/china-completes-durbar-high-school-reconstruction-in-10-months.html', 'Category': ['Heritage', 'Just In', 'Chinese support', '

In [49]:
newsCollection.drop()


In [56]:
 for i in myclient.list_databases():
        print(i)

{'name': 'admin', 'sizeOnDisk': 139264.0, 'empty': False}
{'name': 'breakfast', 'sizeOnDisk': 69632.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 73728.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 106496.0, 'empty': False}
{'name': 'mydatabase', 'sizeOnDisk': 73728.0, 'empty': False}


In [40]:
x = mycol.find_one()
print(x)

{'_id': ObjectId('5ceb501d155c452670eaaa23'), 'name': 'John', 'address': 'Highway 37'}


In [51]:

import pymongo

myc = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myc["mydatabase"]
mycol = mydb["customers"]

mydict = { "name": "John", "address": "Highway 37" }

mycol.insert_one(mydict)

In [50]:
breakfastDB.list_collection_names()

[]

In [57]:
print(x.inserted_id)


AttributeError: 'dict' object has no attribute 'inserted_id'

In [ ]:
#myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#breakfastDB = myclient["breakfast"]
#newsCollection = breakfastDB["newNews"]

def clean_description(description):
    soup_description =BeautifulSoup(description,'html.parser')
    if (soup_description.find('p')==None):
        c_description = soup_description.text
    else:
        c_description = soup_description.find("p").text
    return c_description
    
urls = {
    "onlinekhabar":"http://english.onlinekhabar.com/feed",
    "the himalayan times":"https://thehimalayantimes.com/feed/",
    "setopati":"https://setopati.net/feed"
    }


for url in urls:
    try:
        response =  requests.get(urls[url],headers=header)
        pagesource = response.content
        soup  = BeautifulSoup(pagesource,'xml')
        item_list = soup.find_all('item')

        for post in item_list:

            categorys=[]
            aDic={}

            title = post.find('title').text # getting title
            pubdate = post.find('pubDate').text # getting publication data
            link  = post.find('link').text    # getting link of the article
            
            category_list = post.find_all('category') # category list
            for category in category_list:
                categorys.append(category.text)
            
            description =  post.find('description').text
            soup_description =BeautifulSoup(post.find('description').text,'html.parser')
            # cleaned
            aDic['Publisher']=url
            aDic['Title']=title
            aDic['Publish_Date'] = date_parser.parse(pubdate)
            aDic['Link'] = link
            aDic['Category']=categorys
            aDic['Description']=clean_description(description)
            newsCollection.insert_one(aDic)
    except Exception as e:
        print(e)
        pass




In [64]:
url = "https://www.aljazeera.com/xml/rss/all.xml"
# urls={"Wired":"https://www.wired.com/feed/rss","Aljazeera":"https://www.aljazeera.com/xml/rss/all.xml"}
response  = requests.get(url,headers = header)
pagesource = response.content
soup  = BeautifulSoup(pagesource,'xml')
item_list = soup.find_all('item')
post=item_list[0]

In [65]:
title = post.find('title').text # getting title
pubdate = post.find('pubDate').text # getting publication data
link  = post.find('link').text    # getting link of the article
            
category_list = post.find_all('category') # category list
for category in category_list:
    categorys.append(category.text)
            
description =  post.find('description').text
soup_description =BeautifulSoup(post.find('description').text,'html.parser')
           
description=clean_description(description)

In [70]:
pubdate

'Sun, 26 May 2019 14:18:31 GMT'

In [77]:
date_parser.parse(pubdate)

datetime.datetime(2019, 5, 26, 14, 18, 31, tzinfo=tzutc())

In [82]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

breakfastDB = myclient["breakfast"]
breakfastDB["newNews"].drop()
newsCollection = breakfastDB["newNews"]


def clean_description(description):
    soup_description =BeautifulSoup(description,'html.parser')
    if (soup_description.find('p')==None):
        c_description = soup_description.text
    else:
        c_description = soup_description.find("p").text
    return c_description
    
urls={
    "Wired":"https://www.wired.com/feed/rss",
    "Aljazeera":"https://www.aljazeera.com/xml/rss/all.xml"
    }

for url in urls:
    try:
        response =  requests.get(urls[url],headers=header)
        pagesource = response.content
        soup  = BeautifulSoup(pagesource,'xml')
        item_list = soup.find_all('item')

        for post in item_list:

            categorys=[]
            aDic={}

            title = post.find('title').text # getting title
            pubdate = post.find('pubDate').text # getting publication data
            link  = post.find('link').text    # getting link of the article
            
            category_list = post.find_all('category') # category list
            for category in category_list:
                categorys.append(category.text)
            
            description =  post.find('description').text
            soup_description =BeautifulSoup(post.find('description').text,'html.parser')
            # cleaned
            aDic['Publisher']=url
            aDic['Title']=title
            aDic['Publish_Date'] = date_parser.parse(pubdate)
            aDic['Link'] = link
            aDic['Category']=categorys
            aDic['Description']=clean_description(description)
            newsCollection.insert_one(aDic)
    except Exception as e:
        print(e)
        pass




In [91]:
import schedule
import time

def job():
    print("I'm working...")

schedule.every(3).seconds.do(job)

"""
schedule.every().day.at("00:15").do(main)
schedule.every().day.at("06:15").do(main)
schedule.every().day.at("12:15").do(main)
"""
while True:
    schedule.run_pending()
    time.sleep(1)


I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...
I'm working...


KeyboardInterrupt: 